# This notebook loads knowledge into a Neo4j KG

- Author: Victor Z. Chen (founder@gopeaks.org)
- Date: April 2, 2022

In [1]:
import neo4j
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from IPython.display import display

In [2]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

# Create nodes and edges in pandas df
### Load the full data into a pandas df

In [6]:
d = pd.read_csv("wellbeing_meta.csv")
d.shape

(13637, 41)

In [7]:
nodes_cons = ['v1_name', 'v1_label1', 'v1_label2', 'v1_label3', 
              'v2_name', 'v2_label1', 'v2_label2', 'v2_label3', 
              'source', 'paperid']

nodes_cons_new = ['v_name', 'v_label1', 'v_label2', 'v_label3', 
              'source', 'paperid']

nodes_vars = ['v1', 'v1_description', 'v1_name', 'v1_mean', 'v1_std', 'v1_alpha', 
              'v2', 'v2_description', 'v2_name', 'v2_mean', 'v2_std', 'v2_alpha', 
              'source', 'paperid']

nodes_vars_new = ['v', 'v_description', 'v_name', 'v_mean', 'v_std', 'v_alpha', 
              'source', 'paperid']

edges_cons = ['v1_name', 'v2_name', 
              'source', 'paperid', 
              'theory', 'hypothesis_id', 'hypothesis', 'hypothesis_link']

edges_vars = ['v1', 'v1_description', 'v1_name',
              'v2', 'v2_description', 'v2_name', 
              'source', 'paperid', 
              'sample_size', 'sample_years', 'sample_country', 'sample_industry', 
              'correlation']

### Nodes (Constructs)

In [8]:
c1s = d[[i for i in nodes_cons if 'v1' in i] + ['source', 'paperid']]
c1s.columns = nodes_cons_new
c2s = d[[i for i in nodes_cons if 'v2' in i] + ['source', 'paperid']]
c2s.columns = nodes_cons_new
cs = pd.concat([c1s, c2s], axis = 0).drop_duplicates(subset=['v_name'], keep='first').reset_index()
cs.loc[cs['v_label1']=="e", 'v_label1']='environment'
cs.loc[cs['v_label1']=="p", 'v_label1']='person'
cs.loc[cs['v_label1']=="i", 'v_label1']='interaction'
cs.loc[cs['v_label1']=="1", 'v_label1']='well-being, physical'
cs.loc[cs['v_label1']=="2", 'v_label1']='well-being, economic'
cs.loc[cs['v_label1']=="3", 'v_label1']='well-being, hedonic'
cs.loc[cs['v_label1']=="4", 'v_label1']='well-being, eudaimonic'
cs.loc[cs['v_label1']=="5", 'v_label1']='well-being, social'
cs.loc[cs['v_label1']=="10", 'v_label1']='performance'
cs.loc[cs['v_label3']=="0", 'v_label3']=cs['v_label2']
cs = cs[cs['v_label1']!="0"]
cs = cs[cs['v_label1']!="999"]
cs.shape

(468, 7)

### Nodes (Variables)

In [9]:
v1s = d[[i for i in nodes_vars if 'v1' in i] + ['source', 'paperid']]
v1s.columns = nodes_vars_new
v2s = d[[i for i in nodes_vars if 'v2' in i] + ['source', 'paperid']]
v2s.columns = nodes_vars_new
vs = pd.concat([v1s, v2s], axis = 0).drop_duplicates(subset=['v', 'v_description'], keep='first').reset_index()
vs.shape

(2883, 9)

### Edges (Theories and Hypotheses)

In [10]:
edges_theory = d[[i for i in edges_cons]]
edges_theory = edges_theory[~edges_theory['hypothesis'].isnull()]
edges_theory = edges_theory.drop_duplicates(subset=['v1_name', 'v2_name', 'source', 'paperid', 'theory', 'hypothesis_id'], 
                                            keep='first').reset_index()
edges_theory.shape

(1372, 9)

### Edges (Samples and Evidence)

In [11]:
edges_evidence = d[[i for i in edges_vars]]
edges_evidence = edges_evidence[~edges_evidence['correlation'].isnull()]
edges_evidence = edges_evidence.drop_duplicates(subset=['v1', 'v1_description', 'v1_name',
                                                        'v2', 'v2_description', 'v2_name', 
                                                        'source', 'paperid'], 
                                                keep='first').reset_index()
edges_evidence.shape

(11493, 14)

# Load nodes and edges into a knowledge graph (KG) database
### Open a local Neo4j desktop or AuraDB

In [24]:
driver = neo4j.GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j","GoPeaks"))

In [25]:
session = driver.session(database="neo4j")

### Clean Neo4j, and create nodes

In [26]:
my_neo4j_wipe_out_database()

Create constructs and tag taxonomic labels

In [27]:
for i in cs.reset_index().index:

    nm = str(cs.iloc[i]['v_name'])
    l1 = str(cs.iloc[i]['v_label1'])
    l2 = str(cs.iloc[i]['v_label2'])
    l3 = str(cs.iloc[i]['v_label3'])
    sc = str(cs.iloc[i]['source'])
    pr = str(cs.iloc[i]['paperid'])
    
    tags = "{" + f"name: \"{nm}\", "
    tags += f"label1: \"{l1}\", "
    tags += f"label2: \"{l2}\", "
    tags += f"label3: \"{l3}\", "
    tags += f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\""
    tags += "}"
    
    query = f"CREATE (c: construct {tags})"

    session.run(query)

Create variables and tag statistical information

In [28]:
for i in vs.index:
    
    nm_r = str(vs.iloc[i]['v'])
    nm = str(vs.iloc[i]['v_name'])
    de = str(vs.iloc[i]['v_description'])
    miu = str(vs.iloc[i]['v_mean'])
    std = str(vs.iloc[i]['v_std'])
    alpha = str(vs.iloc[i]['v_alpha'])
    sc = str(vs.iloc[i]['source'])
    pr = str(vs.iloc[i]['paperid'])
    
    tags = "{" + f"name: \"{nm}\", "
    tags += f"name_raw: \"{nm_r}\", "
    tags += f"description: \"{de}\", "
    tags += f"mean: \"{miu}\", "
    tags += f"std: \"{std}\", "
    tags += f"alpha: \"{alpha}\", "
    tags += f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\""
    tags += "}"
    
    query = f"CREATE (v: variable {tags})"

    session.run(query)

### Create edges
Construct-Variable Links

In [29]:
for i in vs.index:
    
    nm_r = str(vs.iloc[i]['v'])
    nm = str(vs.iloc[i]['v_name'])
    de = str(vs.iloc[i]['v_description'])
    miu = str(vs.iloc[i]['v_mean'])
    std = str(vs.iloc[i]['v_std'])
    alpha = str(vs.iloc[i]['v_alpha'])
    sc = str(vs.iloc[i]['source'])
    pr = str(vs.iloc[i]['paperid'])
    
    match = f"WHERE c.name = \"{nm}\" AND v.name = \"{nm}\" "
    match += f"AND v.name_raw = \"{nm_r}\" "
    match += f"AND v.description = \"{de}\""
    
    tags = "{" + f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\""
    tags += "}"
    
    query = f"""
    MATCH (v: variable), (c: construct)
    {match}
    CREATE (v)-[r:measures {tags}]->(c)
    """
    session.run(query)

Hypothesis and Theory

In [30]:
for i in edges_theory.index:
    
    sc = str(edges_theory.iloc[i]['source'])
    pr = str(edges_theory.iloc[i]['paperid'])
    th = str(edges_theory.iloc[i]['theory'])
    hypo_id = str(edges_theory.iloc[i]['hypothesis_id'])
    hypo = str(edges_theory.iloc[i]['hypothesis'])
    hypo_link = str(edges_theory.iloc[i]['hypothesis_link'])
    
    c1nm = str(edges_theory.iloc[i]['v1_name'])
    c2nm = str(edges_theory.iloc[i]['v2_name'])
    
    match = f"WHERE c1.name = \"{c1nm}\" AND c2.name = \"{c2nm}\" "
    match += f"AND c1.source = \"{sc}\" AND c2.source = \"{sc}\" "
    match += f"AND c1.paper_id = \"{pr}\" AND c2.paper_id = \"{pr}\""
    
    tags = "{" + f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\", "
    tags += f"theory: \"{th}\", "
    tags += f"hypothesis_id: \"{hypo_id}\", "
    tags += f"hypothesis: \"{hypo}\", "
    tags += f"hypothesis_link: \"{hypo_link}\""
    tags += "}"
    
    query = f"""
    MATCH (c1: construct), (c2: construct)
    {match}
    CREATE (c1)-[r:causes {tags}]->(c2)
    """
    session.run(query)

Correlations and Sample Statistics

In [31]:
for i in edges_evidence.index:
    
    sc = str(edges_evidence.iloc[i]['source'])
    pr = str(edges_evidence.iloc[i]['paperid'])
    N = str(edges_evidence.iloc[i]['sample_size'])
    time = str(edges_evidence.iloc[i]['sample_years'])
    place = str(edges_evidence.iloc[i]['sample_country'])
    ind = str(edges_evidence.iloc[i]['sample_industry'])
    corr = str(edges_evidence.iloc[i]['correlation'])
    
    v1 = str(edges_evidence.iloc[i]['v1'])
    v2 = str(edges_evidence.iloc[i]['v2'])
    v1nm = str(edges_evidence.iloc[i]['v1_name'])
    v2nm = str(edges_evidence.iloc[i]['v2_name'])
    v1de = str(edges_evidence.iloc[i]['v1_description'])
    v2de = str(edges_evidence.iloc[i]['v2_description'])
    
    match = f"WHERE v1.name_raw = \"{v1}\" AND v2.name_raw = \"{v2}\" "
    match += f"AND v1.name = \"{v1nm}\" AND v2.name = \"{v2nm}\" "
    match += f"AND v1.description = \"{v1de}\" AND v2.description = \"{v2de}\""
    
    tags = "{" + f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\", "
    tags += f"sample_size: \"{N}\", "
    tags += f"sample_years: \"{time}\", "
    tags += f"sample_countries: \"{place}\", "
    tags += f"sample_industries: \"{ind}\", "
    tags += f"correlation: \"{corr}\""
    tags += "}"
    
    query = f"""
    MATCH (v1: variable), (v2: variable)
    {match}
    CREATE (v1)-[r:correlates_with {tags}]->(v2)
    """
    session.run(query)

Correlation is both-way, so duplicate all  the correlations for the other direction again.

In [32]:
for i in edges_evidence.index:
    
    sc = str(edges_evidence.iloc[i]['source'])
    pr = str(edges_evidence.iloc[i]['paperid'])
    N = str(edges_evidence.iloc[i]['sample_size'])
    time = str(edges_evidence.iloc[i]['sample_years'])
    place = str(edges_evidence.iloc[i]['sample_country'])
    ind = str(edges_evidence.iloc[i]['sample_industry'])
    corr = str(edges_evidence.iloc[i]['correlation'])
    
    v1 = str(edges_evidence.iloc[i]['v1'])
    v2 = str(edges_evidence.iloc[i]['v2'])
    v1nm = str(edges_evidence.iloc[i]['v1_name'])
    v2nm = str(edges_evidence.iloc[i]['v2_name'])
    v1de = str(edges_evidence.iloc[i]['v1_description'])
    v2de = str(edges_evidence.iloc[i]['v2_description'])
    
    match = f"WHERE v1.name_raw = \"{v1}\" AND v2.name_raw = \"{v2}\" "
    match += f"AND v1.name = \"{v1nm}\" AND v2.name = \"{v2nm}\" "
    match += f"AND v1.description = \"{v1de}\" AND v2.description = \"{v2de}\""
    
    tags = "{" + f"source: \"{sc}\", "
    tags += f"paper_id: \"{pr}\", "
    tags += f"sample_size: \"{N}\", "
    tags += f"sample_years: \"{time}\", "
    tags += f"sample_countries: \"{place}\", "
    tags += f"sample_industries: \"{ind}\", "
    tags += f"correlation: \"{corr}\""
    tags += "}"
    
    query = f"""
    MATCH (v1: variable), (v2: variable)
    {match}
    CREATE (v2)-[r:correlates_with {tags}]->(v1)
    """
    session.run(query)